In [26]:
import os
import openai
from openai import OpenAI

In [ ]:
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [29]:
response = client.chat.completions.create(
    model="gpt-4o",
    messages=[{"role":"user", "content":"is it too late to join the course?"}]
)

In [30]:
response

ChatCompletion(id='chatcmpl-AW5kNSx9N3A9t3RMLYKmTWzIs9x0Z', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="I would need more information to provide a specific answer. It depends on several factors:\n\n1. **Enrollment Deadlines:** Check the course's official website or contact the institution offering the course to see if enrollment is still open.\n\n2. **Course Start Date:** If the course has already started, find out how far it has progressed and whether late joiners are accommodated.\n\n3. **Course Flexibility:** Some courses are self-paced, meaning you can join at any time, while others follow a strict schedule.\n\n4. **Available Spots:** Ensure there are still open spots available for enrollment in the course.\n\n5. **Prerequisite Requirements:** Consider whether you've met any prerequisite requirements necessary for joining the course at this stage.\n\nIf you provide more details about the specific course, I might be able to he